<a href="https://colab.research.google.com/github/eyash24/Data-Science/blob/main/NLP_101_Fake_News_Classifier_Using_Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake news classifier using Bidirectional LSTM RNN

Following Krish Naik video: https://www.youtube.com/live/gFwr2KxZ1pc?si=k0xvP9aap6N4fW1G

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eyash24","key":"927bd2fd642245307c1a8effaf8ab2b2"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c fake-news

100% 46.5M/46.5M [00:00<00:00, 106MB/s] 
100% 46.5M/46.5M [00:00<00:00, 91.1MB/s]


In [4]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd

df = pd.read_csv('train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.shape

(20800, 5)

In [10]:
df = df.dropna()

In [11]:
df.shape

(18285, 5)

In [12]:
# Separating the independent feature
X = df.drop('label', axis=1)

In [13]:
y = df['label']

In [14]:
# checking whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [15]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [16]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [17]:
# vocabulary size
voc_size = 5000

In [33]:
messages = X.copy()

In [19]:
import nltk

In [20]:
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
X.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [34]:
messages.reset_index(inplace=True)

In [ ]:
# Data preprocessing
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', " ", messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [37]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[2076, 4622, 3820, 4178, 4968, 2472, 4207, 285, 822, 2651],
 [3044, 1128, 2959, 3247, 2272, 2687, 3451],
 [320, 1514, 4563, 4574],
 [4821, 2519, 2459, 558, 345, 494],
 [3104, 2272, 3310, 2441, 4952, 360, 2272, 4678, 907, 2891],
 [2183,
  2819,
  262,
  2350,
  3599,
  3139,
  1430,
  4273,
  923,
  3539,
  4270,
  947,
  4541,
  1044,
  3451],
 [997, 4778, 878, 4491, 4896, 3289, 296, 2493, 2353, 61, 4276],
 [3204, 3466, 4284, 4338, 2301, 2110, 3139, 235, 2353, 61, 4276],
 [265, 3207, 4492, 2108, 1613, 917, 2918, 4494, 3139, 119],
 [3055, 4467, 375, 4876, 4210, 4827, 3225, 3476],
 [2819, 1098, 845, 2232, 2542, 1152, 1157, 1594, 1003, 883, 3471],
 [558, 477, 4968, 917, 3139, 2301],
 [37, 4430, 3355, 4012, 2961, 4712, 1780, 4563, 897],
 [2286, 2263, 4694, 2041, 3908, 816, 4982, 2353, 61, 4276],
 [4949, 4861, 3468, 4173, 4749, 2353, 61, 4276],
 [1216, 958, 3391, 2268, 3825, 37, 100, 4967, 2323, 2899],
 [377, 4519, 1128],
 [3809, 4205, 610, 4233, 3139, 1893, 2024, 3451],
 [4523, 803, 2959,

## Embedding Representation

In [38]:
# max_sentence length
max_len = max([len(word) for word in onehot_repr])

In [39]:
max_len

47

In [41]:
sent_length = 60
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  285,  822, 2651],
       [   0,    0,    0, ..., 2272, 2687, 3451],
       [   0,    0,    0, ..., 1514, 4563, 4574],
       ...,
       [   0,    0,    0, ..., 2353,   61, 4276],
       [   0,    0,    0, ..., 1051,   65, 3151],
       [   0,    0,    0, ..., 3351, 1723, 3642]], dtype=int32)

In [47]:
# creating the model
embedding_vector_features = 80
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 80)            400000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               144800    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 545001 (2.08 MB)
Trainable params: 545001 (2.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [43]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [44]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.3, random_state = 42)

In [48]:
# Model training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=32)

Epoch 1/2
400/400 [==============================] - 58s 135ms/step - loss: 0.2599 - accuracy: 0.8812 - val_loss: 0.1948 - val_accuracy: 0.9156
Epoch 2/2
400/400 [==============================] - 72s 179ms/step - loss: 0.1278 - accuracy: 0.9506 - val_loss: 0.2010 - val_accuracy: 0.9191


In [49]:
y_pred = model.predict(X_test)

172/172 [==============================] - 8s 41ms/step


In [51]:
y_pred = np.where(y_pred >= 0.5, 1, 0)

In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[2803,  304],
       [ 140, 2239]])

In [53]:
print(accuracy_score(y_test, y_pred))

0.9190667152752461
